# Lesson8 单层神经网络

## 一、单层神经网络：线性回归

### 1.单层回归网络的理论基础

在深度学习中，我们用$z$表示线性回归的结果。考虑$m$个样本的线性回归为$\hat{z}=b+w_1x_1,+...+w_nx_n$，矩阵表示如下:
$$\hat{z}=Xw$$

- 单层神经网络
  - 在描述神经网络的层数的时候，我们不考虑输入层。



### 2.tensor实现单层神经网络的正向传播

- `torch.Tensor()` 无论输入数据的类型，都生成的是浮点型数据
- `torch.tensor()` 根据输入的类型，来确定数据类型
  

**tensor计算中的坑**
- PyTorch的静态性
  - torch中要求分类标签必须是整形，不能是浮点型
  - torch有许多函数要求真实标签的类型必须与预测值的类型一致
  - PyTorch中许多函数不接受一维张量但同时也有许多函数不接受二维标签
- 精度问题
  - float32只要保留32位，存在精度问题
  - torch.mv()在计算时，也容易出现精度问题
  - 精度问题在tensor维度很高，数字很大时，也会变得更大
  - PyTorch设置了64位浮点数来尽量减轻精度问题，但是很占内存计算很慢
  - `allclose(x,x_1)` 忽视非常小的区别，判断是否相等



In [11]:
#————————————————————————————单层回归神经网络的简单例子————————————————————————————
import torch

X = torch.tensor([[1,0,0],[1,1,0],[1,0,1],[1,1,1]],dtype=torch.float32)  # 要记得定义标签类型，最安全的就是 先写floa32
z = torch.tensor([-0.2,-0.05,-0.05,0.1])
w = torch.tensor([-0.2,0.15,0.15])

x # 第一列全是1 用与拟合参数b

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 0., 1.],
        [1., 1., 1.]])

In [12]:
X*w

tensor([[-0.2000,  0.0000,  0.0000],
        [-0.2000,  0.1500,  0.0000],
        [-0.2000,  0.0000,  0.1500],
        [-0.2000,  0.1500,  0.1500]])

In [17]:
def LinearR(X,w):
    zhat = torch.mv(X,w)  # mv(矩阵，向量)
    return zhat

In [23]:
zhat = LinearR(X,w)
zhat

tensor([-0.2000, -0.0500, -0.0500,  0.1000])

In [25]:
zhat == z # zhat是一维的，z是二维的

z = z.view([4])
z == zhat  

# 为什么会出现不等于？

tensor([ True, False, False, False])

In [27]:
# 计算SSE，zhat与z相等则SSE应该完全为0
SSE = sum((zhat-z) ** 2)
SSE

tensor(8.3267e-17)

In [28]:
#设置显示精度，再来看yhat与y_reg
torch.set_printoptions(precision=30)  # 查看小数点后30位的情况
zhat  

# zhat 和 z 有精度的差异

tensor([-0.200000002980232238769531250000, -0.049999997019767761230468750000,
        -0.049999997019767761230468750000,  0.100000008940696716308593750000])

In [29]:
# 精度的问题的例子

preds = torch.ones(300,68,64,64)
preds.shape

torch.Size([300, 68, 64, 64])

In [31]:
a = 300*68*64*64
a   # 8kw多个数据

83558400

In [32]:
preds_ = torch.ones(300,68,64,64) * 0.1
preds_.sum() *10

# 83558400 与 tensor(83558328.) 精度出现了问题
# PyTorch设置了64位浮点数来尽量减轻精度问题


tensor(83558328.)

In [34]:
#与Python中存在Decimal库不同，PyTorch设置了64位浮点数来尽量减轻精度问题
preds__ = torch.ones(300, 68, 64, 64, dtype=torch.float64) * 0.1
preds__.sum() * 10


# float64 很占内存，计算速度很慢，有精度要求时使用

tensor(83558400.000000059604644775390625000000, dtype=torch.float64)

In [36]:
#如果你希望能够无视掉非常小的区别，而让两个张量的比较结果展示为True，你可以使用下面的代码：
torch.allclose(zhat,z)


True

### 4.torch.nn.Linear实现单层回归神经网络的正向传播

- `torch.nn.Linear`类来实现单层回归神经网络
  - in_features: int 输入的特征数
  - out_features: int 输出的特征数
  - bias: bool = True 截距
- 属性`.weight` 查看生成的w
- 属性`.bias` 查看生成的截距

- 人为设置随机数种子 保证每次运行的参数都相同torch.random.manual_seed(任意整数)


In [37]:
import torch

# 刷新前面设置的30位小数

In [38]:
X = torch.tensor([[0,0],[1,0],[0,1],[1,1]],dtype=torch.float32)
X


tensor([[0., 0.],
        [1., 0.],
        [0., 1.],
        [1., 1.]])

In [46]:
#————————————————————————————实例化nn.Linear————————————————————————————

# 人为设置随机数种子 保证每次运行的参数都相同
# 随机数种子要和nn.Linear在同一个cell中
torch.random.manual_seed(420)


output = torch.nn.Linear(2,1,bias=True)

# 单层网络 也就输入层和输出层 输入层加权和的结果就是输出
# 上一层的神经元个数（不需要考虑x0 只需要考虑真正的特征）
# 这一层的神经元的个数
# bias 控制是否生成截图

In [48]:
output.weight  # 查看生成的w

Parameter containing:
tensor([[ 0.431836068630218505859375000000, -0.425621807575225830078125000000]],
       requires_grad=True)

In [47]:
output.bias   #  查看截距

Parameter containing:
tensor([0.672958195209503173828125000000], requires_grad=True)

In [43]:
zhat = output(X)
zhat

tensor([[ 0.095281660556793212890625000000],
        [ 0.224004089832305908203125000000],
        [-0.516246497631072998046875000000],
        [-0.387524068355560302734375000000]], grad_fn=<AddmmBackward0>)

- 由于不需要定义常量b，因此在特征张量中，也**不需要留出与常数项相乘的x0**那一列。在输入数据时，我们只输入了两个特征x1与x2
- 输入层只有一层，并且输入层的结构（神经元的个数）由输入的特征张量 决定，因此在PyTorch中构筑神经网络时，不需要定义输入层
- 实例化之后，将特征张量输入到实例化后的类中，即可得出输出层的输出结果。

## 二、二分类神经网络：逻辑回归

### 1.二分类神经网络的理论基础

对线性回归进行一定的变换，称为广义线性回归，例如等式两边同时取对数的对数函数回归、同时取指数的S形函数回归等。
- **Sigmoid函数**$Sigmoid(z)=\frac{1}{1+e^-z}=\sigma$
  - 对线性回归的结果$z$进行变换
  - 有将连续性变量转化为离散型变量，化回归算法为分类算法


- **逻辑回归(对数几率回归)**
  1. 将结果$\sigma$以几率($\frac{\sigma}{1-\sigma}$)的形式展现
  2. 在几率上秋$e$为底的对数
   $\ln{\frac{\sigma}{1-\sigma}}=\ln{e^{Xw}}={Xw}$
  - 虽然叫逻辑回归，但常用于分类问题 

### 2.tensor实现二分类神经网络的正向传播

- 以与门为例

In [49]:
#————————————————————————————以与门为例————————————————————————————
X = torch.tensor([[1,0,0],[1,1,0],[1,0,1],[1,1,1]],dtype=torch.float32)
andgate = torch.tensor([[0],[0],[0],[1]],dtype=torch.float32)
w = torch.tensor([-0.2,0.15,0.15], dtype = torch.float32)

# 保险起见，生成二维的、float32类型的标签

In [50]:
#————————————————————————————定义逻辑回归的函数————————————————————————————
def LogisticR(X,w):
    zhat = torch.mv(X,w)
    sigma = 1/(1+torch.exp(-zhat))
    # 设置阈值为0.5 大于0.5 为1 
    andhat = torch.tensor([int(x) for x in sigma >= 0.5],dtype=torch.float32)
    return sigma,andhat

In [54]:
sigma, andhat = LogisticR(X,w)
sigma  # sigma 可以看做事件发生的概率

tensor([0.450166016817092895507812500000, 0.487502634525299072265625000000,
        0.487502634525299072265625000000, 0.524979174137115478515625000000])

In [53]:
 andhat

tensor([0., 0., 0., 1.])

### 3.符号函数sign, ReLU, Tanh

这些函数是最常见的二分类转化函数，他们在神经网络的结构中有着不可替代的作用。符号和双曲正切函数都从torch中调用

- **符号函数(阶跃函数)**
$$y=\begin{cases}
-1,if \quad x< 0 \\\\
0,if\quad x= 0 \\\\
1,if \quad x>0
\end{cases}$$

- **ReLU(整流线型单元函数)**  
  - ReLU函数现在神经网络领域中的宠儿，应用甚至比sigmoid更广泛。
  - 当输入的自变量大于0时，直接输出该值，当输入的自变量小于等于0时，输出0。本质就是$\max(0,z)$
  - 从nn.functional中调用
$$ReLU:\sigma=\begin{cases}
0,if \quad z\leq 0 \\\\
z,if\quad z> 0 
\end{cases}$$

- **tanh(双曲正切函数)**
  - 双曲正切函数的性质与sigmoid相似，它能够将数值压缩到(-1,1)区间内。
  $$tanh:\sigma=\frac{e^{2z}-1}{e^{2z}+1}$$


  

### 4.torch.functional实现单层二分类神经网络的正向传播

逻辑回归与线性回归的唯一区别，就是在线性回归的结果之后套上了sigmoid函数。
- 从`nn.functional`中调用相关函数
  - 调取的都是函数 不是类

In [55]:
import torch
from torch.nn import functional as F


In [56]:
X = torch.tensor([[0,0],[1,0],[0,1],[1,1]], dtype = torch.float32)


In [60]:
torch.random.manual_seed(420) #人为设置随机数种子  必须和nn.Linear() 放在一个cell里
dense = torch.nn.Linear(2,1)  # 上一层神经元的个数和下一层神经元的个数
zhat = dense(X)
sigma =  F.sigmoid(zhat)  # 调用nn.functionald中的sigmoid函数
y = [int(x) for x in sigma>=0.5]
y

[1, 1, 1, 1]

In [64]:
#————————————————————————————ReLU、Tanh以及Sign函数————————————————————————————
# 符号和双曲正切函数都从torch中调用

torch.sign(zhat)


tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SignBackward0>)

In [66]:
F.relu(zhat)  # max(0,zhat)


tensor([[0.672958195209503173828125000000],
        [1.104794263839721679687500000000],
        [0.247336387634277343750000000000],
        [0.679172456264495849609375000000]], grad_fn=<ReluBackward0>)

In [67]:
torch.tanh(zhat)

tensor([[0.586922407150268554687500000000],
        [0.802214503288269042968750000000],
        [0.242413192987442016601562500000],
        [0.590981125831604003906250000000]], grad_fn=<TanhBackward0>)

## 三、多分类神经网络：Softmax回归

### 1.认识softmax函数

在多分类中，神经元的个数与标签类别的个数是一致的。样本的预测标签就是**所有输出的概率中最大的概率对应的标签类别**
$$\sigma_k=Softmax(z_k)=\frac{e^{z_k}}{\sum^K{e^z}}$$

很容易可以看出，Softmax函数的分子是多分类状况下某一个标签类别的回归结果的指数函数，分母是多分类状况下所有标签类别的回归结果的指数函数之和，因此**Softmax函数的结果代表了样本的结果为类别$k$的概率**。


### 2.PyTorch中的softmax函数

- `torch.softmax(张量z，n)`
  - 第一个参数是我们输入的用来进行计算的张量z，
  - 第二次参数代表你希望运行softmax计算的维度的索引。
    - 0就是由shape返回的第0维

- softmax函数将多分类问题转换为概率，但是计算量十分巨大
  


In [70]:
#————————————————————————————例子：计量算过大 出现溢出————————————————————————————
#对于单一样本，假定一组巨大的z
z = torch.tensor([1010,1000,990], dtype=torch.float32)

# softmax函数的运算
torch.exp(z) / torch.sum(torch.exp(z))  



tensor([nan, nan, nan])

In [71]:
#————————————————————————————直接调用softmax函数————————————————————————————
torch.softmax(z,0)

# 利用函数不容易溢出

tensor([9.999545812606811523437500000000e-01,
        4.539786823443137109279632568359e-05,
        2.061059989344471432559657841921e-09])

In [72]:
#————————————————————————————调小z，查看两者区别————————————————————————————
# 假设三个输出层神经元得到的z分别是10，9，5  
# 调小z
z = torch.tensor([10,9,5], dtype=torch.float32)
torch.exp(z) / torch.sum(torch.exp(z)) # softmax函数的运算

tensor([0.727475106716156005859375000000, 0.267623156309127807617187500000,
        0.004901689011603593826293945312])

In [89]:
z.shape

torch.Size([3])

In [73]:
torch.softmax(z,0)

# 两者结果一样

tensor([0.727475166320800781250000000000, 0.267623156309127807617187500000,
        0.004901689011603593826293945312])

softmax函数输出的是从0到1.0之间的实数，而且多个输出值的总和是1。因为有了这个性质，我们可以把softmax函数的输出解释为“概率”，

In [74]:
s = torch.tensor([[[1,2],[3,4],[5,6]],[[5,6],[7,8],[9,10]]], dtype=torch.float32)
s.shape

torch.Size([2, 3, 2])

In [79]:
s.shape[0]

2

In [78]:
z.shape[0]   #  前面softmax(z,0) 的0在z中就3

3

### 3.练习：使用nn.Linear与functional实现多分类神经网络的正向传播

你需要弄明白这些问题：
1. nn.Linear中需要输入什么结构？
2. softmax中的dim应该填写什么值？
3. 最终输出的sigma的shape应该是几乘几才对？

In [97]:
#————————————————————————————练习：我的答案————————————————————————————
X = torch.tensor([[0,0],[1,0],[0,1],[1,1]], dtype = torch.float32)
dense = torch.nn.Linear(2,1,bias=False)
zhat = dense(X)
sigma = torch.softmax(zhat,0)
zhat  # 设置的nn.Linear结构不对，导致返回的zhat也没有可以求和的维度

# 错误：
# 1. 三分类问题 应该输出三个神经元 返回在3个分类上的概率
# 2. 虽然特征矩阵中没有全为1的一列来拟合参数b，但是ANN中默认会拟合常数项，无须关闭bias
# 3. softmax的维度错误 需要加和的维度是1

tensor([[0.000000000000000000000000000000],
        [0.077730834484100341796875000000],
        [0.574015319347381591796875000000],
        [0.651746153831481933593750000000]], grad_fn=<MmBackward0>)

In [91]:
#————————————————————————————正确答案————————————————————————————
torch.random.manual_seed(420) 
dense = torch.nn.Linear(2,3) #此时，输出层上的神经元个数是3个，因此应该是（2，3）
zhat = dense(X)
sigma = F.softmax(zhat,dim=1) #此时需要进行加和的维度是1
sigma


tensor([[0.462298959493637084960937500000, 0.349371731281280517578125000000,
         0.188329339027404785156250000000],
        [0.459773510694503784179687500000, 0.442208200693130493164062500000,
         0.098018281161785125732421875000],
        [0.489574432373046875000000000000, 0.322912752628326416015625000000,
         0.187512844800949096679687500000],
        [0.490227818489074707031250000000, 0.411511898040771484375000000000,
         0.098260335624217987060546875000]], grad_fn=<SoftmaxBackward0>)

In [92]:
zhat.shape

torch.Size([4, 3])

In [87]:
X.shape # 4个样本2个特征

torch.Size([4, 2])